In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ==========================================
# 환경 설정 및 라이브러리 설치
# ==========================================
#!pip uninstall -y torch torchvision torchaudio #import torch안되서 추가한것
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
!pip install transformers==4.37.2 accelerate pandas tqdm
!pip install transformers torch pandas tqdm
!pip install -q jamo

import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from kobart import get_kobart_tokenizer, get_pytorch_kobart_model
from transformers import BartForConditionalGeneration, AdamW
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from google.colab import drive

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-2772ciy4/kobart_2d292bbf99e748e791c24ddab6da996f
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-install-2772ciy4/kobart_2d292bbf99e748e791c24ddab6da996f
  Resolved https://github.com/SKT-AI/KoBART to commit 79ff1d76ed47550695a05f4cd923642d754b0929
  Preparing metadata (setup.py) ... done
  Created wheel for kobart: filename=kobart-0.5.2-py3-none-any.whl size=8635 sha256=f03ee4453aee29d5b7edf47839317f64dc914c8bbd6f2ea32dba0051a81a2c7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-505tla3b/wheels/f5/70/d7/ae83146ce76b253c2ade2970070d4ae8029b4a2b241f13c3f5
Successfully built kobart
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.2 MB/s eta 0:00:00
  A

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ 사용 중인 디바이스: {device}")

✅ 사용 중인 디바이스: cuda


### 자모 분리 전처리 함수

In [4]:
import re
import random
from jamo import h2j, j2hcj

# ==========================================
# 3. 야민정음 딕셔너리 및 변환 함수 정의
# ==========================================
yamin_dict = {
    # 1. 시각적 유사성 (Traditional Yamin) & rule.md Update
    '대': ['머', '퍼', '떠'],
    '머': ['대', '퍼', '떠'],
    '퍼': ['대', '머', '떠'],
    '떠': ['대', '머', '퍼'],

    '댕': ['멍', '펑', '떵'],
    '멍': ['댕', '펑', '떵'],
    '펑': ['댕', '멍', '떵'],
    '떵': ['댕', '멍', '펑'],

    '댸': ['며', '띠', '펴'],
    '며': ['댸', '띠', '펴'],
    '띠': ['댸', '며', '펴'],
    '펴': ['댸', '며', '띠'],

    '커': ['귀', '긲'],
    '귀': ['커', '긲'],
    '긲': ['커', '귀'],

    '파': ['과', '마', '따'],
    '과': ['파', '마', '따'],
    '마': ['파', '과', '따'],
    '따': ['파', '과', '마'],

    '피': ['끠', '괴', '미'],
    '끠': ['피', '괴', '미'],
    '괴': ['피', '끠', '미'],
    '미': ['피', '끠', '괴'],

    '비': ['네', 'B', '13'],
    '네': ['비'],

    '근': ['ㄹ'],
    'ㄹ': ['근'],

    '위': ['읶'],
    '읶': ['위'],

    '굿': ['긋'],
    '긋': ['굿'],

    '삐': ['볘'],
    '볘': ['삐'],

    '포': ['쪼', '모'],
    '쪼': ['포', '모'],
    '모': ['포', '쪼', '巳'],

    '지': ['거', '겨', '저'],
    '거': ['지', '겨', '저'],
    '겨': ['지', '거', '저'],
    '저': ['지', '거', '겨'],

    '꺼': ['77ㅓ', '刀ㅓ'],

    '공': ['끙'],
    '끙': ['공'],

    '길': ['긛'],
    '긛': ['길'],

    '돼': ['태', 'EH'],
    '태': ['돼', 'EH'],

    '맨': ['괜', '팬', '땐'],
    '괜': ['맨', '팬', '땐'],
    '팬': ['맨', '괜', '땐'],
    '땐': ['맨', '괜', '팬'],

    '의': ['익'],
    '익': ['의'],

    '루': ['륵'],
    '륵': ['루'],

    '빅': ['븨', '넥'],
    '븨': ['빅', '넥'],
    '넥': ['빅', '븨'],

    '주': ['즉'],
    '즉': ['주', 'KIГ'],

    '쥬': ['즊'],
    '즊': ['쥬'],

    # 2. 기타 한글/모양/특수문자 매핑
    '유': ['윾'],
    '윾': ['유'],
    '우': ['윽'],
    '윽': ['우'],
    '식': ['싀'],
    '싀': ['식'],

    '이': ['OI', 'Ol', '01', '0I', '0l', 'ㅇI', 'ㅇl', 'e', '2', 'O1'],
    '병': ['ㅹ'],
    '리': ['21', 'z', 'R1', 'ri'],
    '구': ['ㅋ', 'ヲ', '7'],
    '기': ['71'],
    '왕': ['앟', '왐'],
    '왱': ['앻'],
    '욍': ['잏'],
    '방': ['bang'],
    '티': ['더', 'T', 't'],
    '도': ['든', 'ㅌ', 'E', 'do'],
    '또': ['SE'],
    '야': ['OF', 'Oㅑ', 'oㅑ'],
    '백': ['뿌'],
    '무': ['ㅁ'],
    '스': ['^', 's'],
    '사': ['ㅅr', '4', 'sa', 'ㅆ'],
    '응': ['%', '÷'],
    '자': ['ㅈr'],
    '가': ['㉠r', 'Lr', '7r', 'go'],
    '나': ['㈁ㅏ', 'Lㅏ', 'na'],
    '다': ['㉰', 'ㅁ-'],
    '라': ['ㄹr', '2r'],
    '바': ['ㅂr', 'b'],
    '아': ['Oㅏ'],
    '차': ['ㅊr'],
    '카': ['ㅋr'],
    '타': ['ㅌr'],
    '하': ['ㅎr', 'Ha'],
    '냐': ['Lㅑ'],
    '샤': ['ㅅㅑ', '^ㅑ'],
    '니': ['L|', 'L1', 'ni', 'u'],
    '디': ['D', 'ㅁ'],
    '시': ['ㅅl', 'tl'],
    '치': ['ㅊl'],
    '키': ['ㅋl', 'key'],
    '히': ['ㅎl'],
    '애': ['OH', 'OII'],
    '개': ['7H'],
    '내': ['Lㅐ', 'LH'],
    '래': ['2H'],
    '재': ['JH', 'H'],
    '에': ['O||'],
    '게': ['7||', 'gay'],
    '세': ['tP', 'say'],
    '요': ['yO', '8', 'dD'],
    '로': ['ro', '2o'],
    '는': ['=','는'],
    '들': ['8', 'dl'],
    '해': ['ㅎri', 'hri', 'ho'],
    '여': ['oㅕ'],
    '음': ['m'],
    '질': ['zl'],
    '읔': ['uk'],
    '와': ['oa', '0a'],
    '소': ['Show', '솓'],
    '고': ['ヱ', '그', '끄'],
    '민': ['괸', '끤'],
    '정': ['경'],
}

# --- 변환 함수 (데이터 증강용) ---
def yaminize(text, prob=0.5):
    """
    text: 원본 문자열
    prob: 각 글자가 변환될 확률 (0.0 ~ 1.0)
    """
    result = []
    for char in text:
        if char in yamin_dict and random.random() < prob:
            replacements = yamin_dict[char]
            if isinstance(replacements, list):
                result.append(random.choice(replacements))
            else:
                result.append(replacements)
        else:
            result.append(char)
    return "".join(result)

# ==========================================
# 전처리 함수: Dictionary에 있는 문자는 분리 X
# ==========================================
# 보존할 문자 집합 생성 (사전에 등장하는 모든 한글 음절)
yamin_preserve_set = set()
for k, v_list in yamin_dict.items():
    if "가" <= k <= "힣":
        yamin_preserve_set.add(k)
    for v in v_list:
        # 한글 음절인 경우만 보존 (그 외 특수문자/영어 등은 어차피 분리 안됨)
        if len(v) == 1 and "가" <= v <= "힣":
            yamin_preserve_set.add(v)

def to_jamo_string(s: str) -> str:
    s = "" if s is None else str(s)
    out = []
    for ch in s:
        # 1. 야민정음 관련 문자(Dictionary에 있는 문자)는 자모 분리 하지 않음 -> 토큰 의미 보존
        if ch in yamin_preserve_set:
            out.append(ch)
        # 2. 그 외 일반 한글 음절만 자모로 분해
        elif "가" <= ch <= "힣":
            out.append(j2hcj(h2j(ch)))
        # 3. 영어, 숫자, 특수문자 등은 그대로 유지
        else:
            out.append(ch)
    return "".join(out)

print(f"✅ 야민정음 사전 로드 완료. 보존할 문자 수: {len(yamin_preserve_set)}개")

✅ 야민정음 사전 로드 완료. 보존할 문자 수: 139개


In [5]:
# ===============================================
# 1. 데이터 불러오기 및 분리 (Train / Test Split)
# ===============================================
path2 = '/content/drive/MyDrive/P01_Nandoc/dataset/yamin_11k.csv'
path3 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_yamin_only.csv'

print("📂 데이터를 불러오는 중입니다...")

# 파일 존재 여부 확인 및 로드
if os.path.exists(path2) and os.path.exists(path3):
    df2 = pd.read_csv(path2)
    df3 = pd.read_csv(path3)

    # 필요한 컬럼만 추출
    df2 = df2[['input', 'output']]
    df3 = df3[['input', 'output']]

    # 데이터 합치기 (샘플링 없이 전체 사용)
    df = pd.concat([df2, df3], axis=0).reset_index(drop=True)

    print("✅ 데이터 병합 완료")
    print(f"총 데이터 수: {len(df)}")
    print("구성 비율:")
    print(f" - df2 (yamin_11k): {len(df2)}")
    print(f" - df3 (new_train): {len(df3)}")

else:
    print("❌ 데이터 파일 경로를 확인해주세요.")
    if not os.path.exists(path2):
        print(f"   - 없음: {path2}")
    if not os.path.exists(path3):
        print(f"   - 없음: {path3}")

📂 데이터를 불러오는 중입니다...
✅ 데이터 병합 완료
총 데이터 수: 16144
구성 비율:
 - df2 (yamin_11k): 11238
 - df3 (new_train): 4906


### 케이스 B
input만 자모로, output은 원문 그대로
“자모 입력을 보고 원문 문장 생성”을 학습

In [6]:
df["input"] = df["input"].apply(to_jamo_string)
# df["output"]은 그대로 둠

In [7]:
# 전처리 결과 샘플 확인 (랜덤 3개)
import random

# 데이터프레임 인덱스 범위 내에서 랜덤 추출
indices = random.sample(range(len(df)), 3)

for idx in indices:
    print(f"--- Sample {idx} ---")
    print("원문(Input) :", df.loc[idx, "input"])
    print("자모(Jamo)  :", to_jamo_string(df.loc[idx, "input"]))
    print("정답(Output):", df.loc[idx, "output"])
    print("")

--- Sample 4004 ---
원문(Input) : ㅍㅕㄴㅇㅏㄴ하게 ㅈㅏㄹ ㅂㅗ내ヱ ㅇㅘㅆㅇㅓ요.
자모(Jamo)  : ㅍㅕㄴㅇㅏㄴ하게 ㅈㅏㄹ ㅂㅗ내ヱ ㅇㅘㅆㅇㅓ요.
정답(Output): 편안하게 잘 보내고 왔어요.

--- Sample 14029 ---
원문(Input) : ㅇㅗㄴㅡㄹ 0lㄱㅗㅅO|| Eㅊㅏㄱㅎㅐㅆ는ㄷㅔ, 마ㅊl ㅅlㄱㅖgo ㅁㅓㅁㅊㅜㄴ ㄷㅡㅅㅎㅏㄴ ㅈㅗㅇㅛㅇㅎㅏㅁ이 goㅈㅏㅇ ㅋㅡㄴ ㅁㅐㄹㅕㄱㅇㅣㄴ ㄱㅓㅅ ㄱㅏㅌOㅏ. 즉ㅂㅕㄴㅇㅡㄴ ㅅㅏㄴㅊㅐㄱro와 ㅂr㉰ㄲㅏ저 10ㅂㅜㄴ 겨R1ㄹrㅅㅓ 아ㅊㅣㅁ과㉰ ㅁㅜㄹ소리 8ㅇㅡ펴 ㅇㅣㄹㅇㅓLㅏㅁㅕㄴ Ha륵㉠r ㅂㅏㄴㅉㅏㄱㅇI지든. ㄹㅜㅁㅇㅡㄴ 20ㅍㅕㅇ0l거ㅁㅏㄴ 7r구㉠r ㄴㅓㅁ 커oㅕㅇㅝㅅㅓ ‘^ㅌㅠㅁㅇㅗㅎㅕㅇ’0l라 ㅂㅜㄹㅡㄹ ㅅㅜ ㅇㅣㅆㄱㅔㅆ거. 에ㅇㅓㅋㅓㄴㅇㅡㄴ ㅇㅓㅄㅇㅓㅆ는ㄷㅔ ㅈㅏㄱㅇㅡㄴ ㅅㅓㄴㅍㅜㅇ71 하나ㅁㅏㄴ ㅇㅣㅆㅇㅡㅁㅕㄴ 30도든 팬ㅊㅏㄶOㅏ. ㉠rㄱㅕㄱㅇㅡㄴ 1ㅂㅏㄱ 15,000ㅇㅝㄴ이라ㅅㅓ ㅈㅓㄹ머 방ㅊlㅎㅏㄹ ㅅㅜ ㅇㅓㅄ는 경doOㅑ. ㅁ-ㅁㅏㄴ sayㅁㅕㄴ도구는 71ㅂㅗㄴㅁㅏㄴ ㅈㅔ끙ㅎr끄, ㅇㅛㄱㅅㅣㄹㅇl ㅈㅗㅂ아 ㅁㅜㄹㅇㅡㄹ 뿌zㅁㅕㄴ 주ㅂㅕㄴㄲㅏ지 스ㅍㅡㄹㅔ2Lr ㄴㅏㄹ아0aㅅㅓ 주익 ㅍㅣㄹ요. Oㅏㅊㅣㅁㅇㅡㄴ ㅅㅔㄹㅍㅡ ㅂㅟㅍㅔ라ㅅㅓ ㅎriㅁㅜㄹ과ㅈㅓㄴ파 ㄱㅣㅁㅊlㅉㅣ7H가 ㅂㅕㄹ피ㅇㅕㅆm. gayㅁ-7r 바ro ㅇㅕㅍ에 24시ㄱㅏㄴ 카ㅍㅔ가 ㅇㅣㅆㅇㅓㅅㅓ ㅂㅏㅁ에 귀미 ㅎㅏㄴㅈㅏㄴHa고 ㅇㅣㄴㅅㅐㅇ 4ㅈㅣㄴ ㅉㅣㄱ기 ㄸㅏㄱ ㅈㅗㅎㅇㅏㅆㅇㅓ. ㅈㅓㄴㅂㅏㄴㅈㅓㄱㅇㅡro ㉠rㅅㅓㅇ13 ㅊㅚ그, ㅁ-ㅁㅏㄴ bang이 ㅈㅗㄱㅡㅁ ㅈㅏㄱ아 지ㅅㅣㄹㅇㅡㄹ 끙윾ㅎrioㅑ hoㅅㅓ 7Hㅇㅣㄴ0l ㅅㅣㅀㅁ-ㅁㅕㄴ 다ㄹㅡㄴ ㅅㅜㄱ소ㄹㅡㄹ ㅊㅏㅈOㅏㅂㅗ는 게 ㅈㅗㅎㄱㅔㅆㅇㅓ.
자모(Jamo)  : ㅇㅗㄴㅡㄹ 0lㄱㅗㅅO|| Eㅊㅏㄱㅎㅐㅆ는ㄷㅔ, 마ㅊl ㅅlㄱㅖgo ㅁㅓㅁㅊㅜㄴ ㄷㅡㅅㅎㅏㄴ ㅈㅗㅇㅛㅇㅎㅏㅁ이 goㅈㅏㅇ ㅋㅡㄴ ㅁㅐㄹㅕㄱㅇㅣㄴ ㄱㅓㅅ ㄱㅏㅌOㅏ. 즉ㅂㅕㄴㅇㅡㄴ ㅅㅏㄴㅊ

In [8]:
# ---------------------------------------------------------
# Train(90%) / Test(10%) 데이터 분리
# ---------------------------------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 인덱스 재정렬
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"✅ 전체 데이터 개수: {len(train_df) + len(test_df)}개")
print(f"   - 학습용 데이터(Train): {len(train_df)}개")
print(f"   - 테스트용 데이터(Test): {len(test_df)}개")

✅ 전체 데이터 개수: 16144개
   - 학습용 데이터(Train): 12915개
   - 테스트용 데이터(Test): 3229개


In [9]:
# ==========================================
# 2. 데이터셋 및 모델 설정
# ==========================================
class KoBARTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input_text = str(row['input'])
        target_text = str(row['output'])

        inputs = self.tokenizer(input_text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        targets = self.tokenizer(target_text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze(),
            'raw_input': input_text,
            'raw_output': target_text
        }

print("🔄 토크나이저 및 모델 로딩 중...")
tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())
model.to(device)

# 학습용 데이터셋 생성
train_dataset = KoBARTDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 테스트용 데이터셋 생성
test_dataset = KoBARTDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

🔄 토크나이저 및 모델 로딩 중...
downloading model from https://huggingface.co/skt/kobart-base-v1/resolve/main/legacy/kobart_base_tokenizer_cased_cf74400bce.zip...
downloading model from https://huggingface.co/skt/kobart-base-v1/resolve/main/legacy/kobart_base_cased_ff4bda5738.zip...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [10]:
# ==========================================
# 3. 학습 (Train Loop)
# ==========================================
optimizer = AdamW(model.parameters(), lr=3e-5)
epochs = 5

print(f"🚀 학습 시작 (총 {epochs} Epochs)...")
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        raw_texts = list(batch["raw_input"]) # 배치의 원문 리스트
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} 평균 Loss: {total_loss / len(train_loader):.4f}")

🚀 학습 시작 (총 5 Epochs)...


Epoch 1/5: 100%|██████████| 808/808 [03:13<00:00,  4.17it/s, loss=0.467]


Epoch 1 평균 Loss: 1.2755


Epoch 2/5: 100%|██████████| 808/808 [03:12<00:00,  4.19it/s, loss=0.061]


Epoch 2 평균 Loss: 0.4106


...

Epoch 3 평균 Loss: 0.2390


Epoch 4/5: 100%|██████████| 808/808 [03:12<00:00,  4.19it/s, loss=0.0274]


Epoch 4 평균 Loss: 0.1874


Epoch 5/5: 100%|██████████| 808/808 [03:12<00:00,  4.19it/s, loss=0.29]

Epoch 5 평균 Loss: 0.1603


In [11]:
# ==========================================
# 4. 모델 저장 및 테스트 결과 확인
# ==========================================
# 1) 학습된 모델 저장
model_save_path = '/content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"💾 모델 저장 완료: {model_save_path}")

# 2) 테스트 데이터(Test Set)로 결과 생성
print(f"📝 테스트 데이터({len(test_df)}개)에 대한 결과 생성 중...")
model.eval()

result_data = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Inference"):
        input_ids = batch['input_ids'].to(device)

        raw_texts = list(batch["raw_input"])

        # 모델 생성 (Beam Search)
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=batch["attention_mask"].to(device),
            max_length=128,
            num_beams=5,
            eos_token_id=tokenizer.eos_token_id
        )
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        for i in range(len(generated_texts)):
            result_data.append({
                "input": batch['raw_input'][i],         # 원본 입력 (야민정음 등)
                "ground_truth": batch['raw_output'][i], # 정답 (원래 문장)
                "prediction": generated_texts[i]        # 모델 예측
            })

# 결과 저장
df_result = pd.DataFrame(result_data)
output_path = "/content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/output_test_results.csv"
df_result.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ 모든 과정 완료! 테스트 결과 파일: {output_path}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


💾 모델 저장 완료: /content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/
📝 테스트 데이터(3229개)에 대한 결과 생성 중...


Inference: 100%|██████████| 101/101 [05:47<00:00,  3.44s/it]

✅ 모든 과정 완료! 테스트 결과 파일: /content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/output_test_results.csv


### 임의 테스트

In [12]:
from transformers import PreTrainedTokenizerFast

# 저장된 모델 경로 (학습 때 저장한 경로와 같아야 합니다)
model_path = '/content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/'

print(f"📂 저장된 모델을 불러오는 중입니다... ({model_path})")

if os.path.exists(model_path):
    # 토크나이저 & 모델 불러오기
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
    model = BartForConditionalGeneration.from_pretrained(model_path)

    # GPU 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # 평가 모드 설정
    print("✅ 모델 로딩 완료! 이제 텍스트를 입력해보세요.")
else:
    print(f"❌ 모델 파일이 없습니다: {model_path}")
    print("   경로를 확인하거나, 학습 코드를 먼저 실행해서 모델을 저장해주세요.")

# ==========================================
# 3. 사용자 입력 및 복원 실행
# ==========================================
def restore_text(text):
    # 입력 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="pt", max_length=256, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 모델 생성 (Generate)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=256,           # 긴 문장도 끊기지 않도록 길이 확보
            num_beams=5,              # 빔 서치 (정확도 향상)
            eos_token_id=tokenizer.eos_token_id,
            bad_words_ids=[[tokenizer.unk_token_id]] # unknown 토큰 생성 방지
        )

    # 결과 디코딩
    result = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return result

# 실행 루프
print("\n" + "="*50)
print("💡 [야민정음/오타 복원기] 시작 (종료: 'q' 입력)")
print("="*50)

while True:
    input_text = input("\n📝 문장을 입력하세요: ")

    if input_text.lower() in ['q', 'quit', 'exit']:
        print("👋 프로그램을 종료합니다.")
        break

    if not input_text.strip():
        continue

    # 복원 수행 및 출력
    restored = restore_text(input_text)
    print(f"▶️ 원본: {input_text}")
    print(f"✨ 복원: {restored}")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


📂 저장된 모델을 불러오는 중입니다... (/content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/)
✅ 모델 로딩 완료! 이제 텍스트를 입력해보세요.

💡 [야민정음/오타 복원기] 시작 (종료: 'q' 입력)

📝 문장을 입력하세요: 숙솞 넚몺 좂ㅇr썺요ㅠㅠ 뷰 ㅁr찞넴
▶️ 원본: 숙솞 넚몺 좂ㅇr썺요ㅠㅠ 뷰 ㅁr찞넴
✨ 복원: 숙소 유리 유리창요 뷰 라이트

📝 문장을 입력하세요: 숙소 너무 좋았어yo ㅠㅠ 뷰 맛있음!
▶️ 원본: 숙소 너무 좋았어yo ㅠㅠ 뷰 맛있음!
✨ 복원: 숙소 너무 좋았어요  뷰 맛있음!!!

📝 문장을 입력하세요: 별 한 7Hdo Oㅏ깝ㅁ-. 왜 ㅆ람80l 별 1개를 주는저 겪어본 sa람으로서 말2o 설명HaㅈrL1 댓글2o는 너ㅁ 긛ヱ... 아ㅁ튼 두 번 다시 Lr긛 싫은 곳. 캠핑을 20oㅕ 년 ㅁ-녀본 곳 중 제일 71분 나빴던 곳.
▶️ 원본: 별 한 7Hdo Oㅏ깝ㅁ-. 왜 ㅆ람80l 별 1개를 주는저 겪어본 sa람으로서 말2o 설명HaㅈrL1 댓글2o는 너ㅁ 긛ヱ... 아ㅁ튼 두 번 다시 Lr긛 싫은 곳. 캠핑을 20oㅕ 년 ㅁ-녀본 곳 중 제일 71분 나빴던 곳.
✨ 복원: 별 한 개도 아깝다. 왜 사진이 별 1개를 주는지 겪어본 사람으로서 말로 설명하니 댓글로는 너무 길고... 암튼 두 번 다시 가기 싫은 곳. 캠핑을 20개월 다녀본 곳 중 제일 기분 나빴던 곳.

📝 문장을 입력하세요: Oㅏ... go격 좋ヱ 뷰do 뻥 뚫려서 ㅅl원Ha거만 담배 냄새 끠쳐버림. 싸gay 하루만 묵겠다! ㅎr는 ㅆ람한테만 추천. 담배 냄새Lr 모든 장점을 ㉠r져Lr는 곳. 노2Hbang에서 각종 담배와 유흥O|| 쩔었을 때 나는 냄새가 계속 bangO|| 있음 ㅆ... 싸니까 할 말 없음.
▶️ 원본: Oㅏ... go격 좋ヱ 뷰do 뻥 뚫려서 ㅅl원Ha거만 담배 냄새 끠쳐버림. 싸gay 하루만 묵겠다! ㅎr는 ㅆ람한테만 추천. 담배 냄새Lr 모든 장점을 ㉠r져Lr는 곳

KeyboardInterrupt: Interrupted by user

In [13]:
# ==========================================
# 5. 성능 평가 (BERTScore & CER)
# ==========================================
!pip install bert_score jiwer

import pandas as pd
from bert_score import score
from jiwer import cer
import numpy as np

# 저장된 결과 파일 불러오기
output_path = "/content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/output_test_results.csv"
if os.path.exists(output_path):
    df_result = pd.read_csv(output_path)

    # 결측치 제거 (문자열이 아닌 경우 방지)
    df_result = df_result.dropna(subset=['ground_truth', 'prediction'])

    preds = df_result['prediction'].astype(str).tolist()
    refs = df_result['ground_truth'].astype(str).tolist()

    # 1. CER (Character Error Rate) 계산
    cer_val = cer(refs, preds)
    print(f"\n📊 CER (Character Error Rate): {cer_val:.4f}")
    print("   (낮을수록 좋음, 0에 가까울수록 정답과 일치)")

    # 2. BERTScore 계산
    print("\n⏳ BERTScore 계산 중... (모델 다운로드로 시간이 소요될 수 있습니다)")
    P, R, F1 = score(preds, refs, lang="ko", verbose=True)

    print(f"📊 BERTScore - Precision: {P.mean().item():.4f}")
    print(f"📊 BERTScore - Recall:    {R.mean().item():.4f}")
    print(f"📊 BERTScore - F1:        {F1.mean().item():.4f}")
    print("   (높을수록 좋음, 1에 가까울수록 문맥적 유사성 높음)")
else:
    print(f"❌ 결과 파일이 존재하지 않습니다: {output_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 106.7 MB/s eta 0:00:00

📊 CER (Character Error Rate): 0.2388
   (낮을수록 좋음, 0에 가까울수록 정답과 일치)

⏳ BERTScore 계산 중... (모델 다운로드로 시간이 소요될 수 있습니다)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/86 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/51 [00:00<?, ?it/s]

done in 12.90 seconds, 250.30 sentences/sec
📊 BERTScore - Precision: 0.9739
📊 BERTScore - Recall:    0.9585
📊 BERTScore - F1:        0.9657
   (높을수록 좋음, 1에 가까울수록 문맥적 유사성 높음)
